# Intent classification

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import spacy

In [18]:
nlp = spacy.load('en_core_web_lg')

In [12]:
df_train=pd.read_table('./datasets/KL/AskUbuntu/train.csv', header=None)
df_test=pd.read_table('./datasets/KL/AskUbuntu/test.csv', header=None)

In [87]:
df_train[1].value_counts()
df_test[1].value_counts()

Software Recommendation    40
Make Update                37
Shutdown Computer          14
Setup Printer              13
None                        5
Name: 1, dtype: int64

In [149]:
vectorizer = CountVectorizer(analyzer = "word", strip_accents=None, tokenizer = None, \
                             preprocessor = None, stop_words = None, max_features = 5000, ngram_range=(1, 3)) 
data_features_train = vectorizer.fit_transform(df_train[0])
tfidfier = TfidfTransformer()
tfidf_train = tfidfier.fit_transform(data_features_train)
data_features_test = vectorizer.transform(df_test[0])
tfidf_test=tfidfier.transform(data_features_test)

In [150]:
cls=LogisticRegression()
cls.fit(tfidf_train, df_train[1])

C:\Users\likkhian\Anaconda3\envs\py3env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\likkhian\Anaconda3\envs\py3env\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [151]:
print(f1_score(df_test[1], cls.predict(tfidf_test), average='micro'), 
     precision_score(df_test[1], cls.predict(tfidf_test), average='micro'), 
     recall_score(df_test[1], cls.predict(tfidf_test), average='micro'))

0.8440366972477065 0.8440366972477065 0.8440366972477065


In [158]:
nlp.add_pipe(nlp.create_pipe('textcat', exclusive_classes=True))

In [162]:
nlp('hello', [1])

TypeError: 'bool' object is not callable

In [159]:
df_train[2]=df_test.apply(lambda x: nlp(x[0]), axis=1)
df_test[2]=df_test.apply(lambda x: nlp(x[0]), axis=1)

TypeError: ("'bool' object is not callable", 'occurred at index 0')

In [153]:
vector_train=np.empty([len(df_train), 300])
for i in range(len(df_train)):
    vector_train[i, :]=df_train.iloc[i, 2].vector
vector_test=np.empty([len(df_test), 300])
for ii in range(len(df_test)):
    vector_test[ii, :]=df_test.iloc[ii, 2].vector

In [156]:
cls=RandomForestClassifier()
cls.fit(vector_train, df_train[1])
print(f1_score(df_test[1], cls.predict(vector_test), average='micro'), 
     precision_score(df_test[1], cls.predict(vector_test), average='micro'), 
     recall_score(df_test[1], cls.predict(vector_test), average='micro'))

C:\Users\likkhian\Anaconda3\envs\py3env\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.23853211009174313 0.23853211009174313 0.23853211009174313
